In [1]:
import os
import glob
import numpy as np
import json
import faiss
import torch
import pandas as pd
from torch import Tensor
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Tuple, Dict, Any

In [2]:
class DocumentRetriever:
    def __init__(self, folder_path: str, benchmark_path: str, retriever_model: str = "intfloat/multilingual-e5-large", batch_size: int = 12):
        self.folder_path = folder_path
        self.benchmark_path = benchmark_path
        self.retriever_model = retriever_model
        self.batch_size = batch_size 

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        self.tokenizer = AutoTokenizer.from_pretrained(retriever_model)
        self.model = AutoModel.from_pretrained(retriever_model).to(self.device)
        self.model.eval()
        
        self.chunks: List[Tuple[str, str]] = []
        self.index = None
        self._text_splitter = self._create_text_splitter()

    def _create_text_splitter(self):
        return RecursiveCharacterTextSplitter(
            chunk_size=300,
            chunk_overlap=50,
            length_function=self._token_counter,
            separators=["\n\n", "\n", "."]
        )

    def _token_counter(self, text: str) -> int:
        return len(self.tokenizer.encode(text, add_special_tokens=False))

    def _read_md_files(self) -> List[Tuple[str, str]]:
        files = glob.glob(os.path.join(self.folder_path, "*.md"))
        documents = []
        for file_path in files:
            with open(file_path, "r", encoding="utf-8") as f:
                doc_name = os.path.splitext(os.path.basename(file_path))[0]
                documents.append((doc_name, f.read()))
        return documents

    def _split_documents(self, documents: List[Tuple[str, str]]):
        self.chunks = []
        for doc_name, content in documents:
            for chunk in self._text_splitter.split_text(content):
                self.chunks.append((doc_name, chunk))

    @staticmethod
    def _average_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
        last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
        return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

    def _create_embeddings(self, texts: List[Tuple[str, str]], is_queries: bool = False) -> np.ndarray:
        embeddings = []
        
        with torch.no_grad():
            for i in tqdm(range(0, len(texts), self.batch_size)):
                batch = texts[i:i+self.batch_size]

                if is_queries:
                    prefixed_batch = batch
                else:
                    prefixed_batch = [f"{doc_name}: {text}" for doc_name, text in batch]

                inputs = self.tokenizer(
                    prefixed_batch,
                    padding=True,
                    truncation=True,
                    return_tensors="pt",
                    max_length=512
                ).to(self.device)

                outputs = self.model(**inputs)
                batch_embeddings = self._average_pool(outputs.last_hidden_state, inputs['attention_mask'])
                batch_embeddings = torch.nn.functional.normalize(batch_embeddings, p=2, dim=1)
                embeddings.append(batch_embeddings.cpu().numpy())
        
        return np.concatenate(embeddings, axis=0)

    def build_index(self):
        documents = self._read_md_files()
        self._split_documents(documents)
        embeddings = self._create_embeddings(self.chunks)
        
        dimension = embeddings.shape[1]
        self.index = faiss.IndexFlatIP(dimension)
        self.index.add(embeddings.astype(np.float32))

    def search(self, df: pd.DataFrame, top_k: int) -> List[Dict[str, Any]]:
        if self.index is None:
            raise ValueError("Индекс не построен. Сначала вызовите build_index()")

        query_embeddings = self._create_embeddings(df['Вопрос'].tolist(), is_queries=True)
        scores, indices = self.index.search(query_embeddings.astype(np.float32), top_k)
        
        results = []
        for i in range(len(df)):
            row = df.iloc[i]
            query_scores = scores[i]
            query_indices = indices[i]
            
            result = {
                "table_data": {
                    "Домен документов": row["Домен документов"],
                    "Сет документов": row["Сет документов"],
                    "Название документа": row["Название документа"],
                    "Отрывок из документа": row["Отрывок из документа"],
                    "Тип вопроса": row["Тип вопроса"],
                    "Вопрос": row["Вопрос"],
                    "Ответ": row["Ответ"]
                },
                "context": []
            }
            
            for idx, score in zip(query_indices, query_scores):
                doc_name, chunk_text = self.chunks[idx]
                result["context"].append({
                    "score": float(score),
                    "doc": doc_name,
                    "text": chunk_text
                })
            
            results.append(result)
        
        return results

    def read_benchmark(self) -> pd.DataFrame:
        set_name = os.path.basename(self.folder_path.rstrip("/\\"))
        df = pd.read_csv(self.benchmark_path)
        return df[df["Сет документов"] == set_name].reset_index(drop=True)

    def search_for_benchmark(self, top_k=5) -> Tuple[pd.DataFrame, List[Dict[str, Any]]]:
        self.build_index()

        df = self.read_benchmark()
        results = self.search(df, top_k)
        return df, results

In [ ]:
all_results = []

base_folder = "../md_benchmark/benchmark"
benchmark_path = "../benchmark/benchmark.csv"

for folder_name in os.listdir(base_folder):
    folder_path = os.path.join(base_folder, folder_name)

    if os.path.isdir(folder_path):
        retriever = DocumentRetriever(
            folder_path=folder_path,
            benchmark_path=benchmark_path,
            batch_size=64
        )
        df, results = retriever.search_for_benchmark(top_k=10)
        all_results.extend(results)

2025-05-11 21:17:11.212501: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746987431.234808  142112 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746987431.241458  142112 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746987431.260660  142112 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746987431.260680  142112 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746987431.260683  142112 computation_placer.cc:177] computation placer alr

In [ ]:
len(all_results)

674

In [ ]:
with open('benchmark.json', 'w', encoding='utf-8') as f:
    json.dump(all_results, f)

In [1]:
results[0]

NameError: name 'results' is not defined

In [ ]:
# retriever = DocumentRetriever(
#     folder_path="../md_benchmark/benchmark/Руководства к РФ ПО",
#     benchmark_path="../benchmark/benchmark.csv"
# )
# df, results = retriever.search_for_benchmark(top_k=10)

2025-05-11 19:39:36.896946: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746981576.916439   87546 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746981576.922924   87546 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746981576.937985   87546 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746981576.937998   87546 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746981576.938001   87546 computation_placer.cc:177] computation placer alr

In [4]:
df

,Домен документов,Сет документов,Название документа,Отрывок из документа,Тип вопроса,Вопрос,Ответ
0,Техническая документация,Руководства к РФ ПО,1c.pdf,Функционирование системы «1С:Предприятие» дели...,Simple,Как делится функционирование системы «1С:Предп...,Функционирование системы «1С:Предприятие» дели...
1,Техническая документация,Руководства к РФ ПО,1c.pdf,Функционирование системы «1С:Предприятие» дели...,With errors,"Как фукционерует системма «1С:Преприятие», на ...",Функционирование системы «1С:Предприятие» дели...
2,Техническая документация,Руководства к РФ ПО,1c.pdf,Функционирование системы «1С:Предприятие» дели...,Trash,"Привет! Мне надо понять, как именно работает э...",Функционирование системы «1С:Предприятие» дели...
3,Техническая документация,Руководства к РФ ПО,1c.pdf,Функционирование системы «1С:Предприятие» дели...,Reformulation,Какие ключевые стадии определяют процесс работ...,Функционирование системы «1С:Предприятие» дели...
4,Техническая документация,Руководства к РФ ПО,1c.pdf,Функционирование системы «1С:Предприятие» дели...,Incorrect by design,В «1С:Предприятие» пользователь сначала работа...,"Нет, наоборот. Функционирование системы «1С:Пр..."
...,...,...,...,...,...,...,...
72,Техническая документация,Руководства к РФ ПО,1c.pdf,Если один и тот же объект пытаются отредактиро...,Logical thinking,"Можно ли провести документ, если он заблокиров...",Нет. Если документ заблокирован другим пользов...
73,Техническая документация,Руководства к РФ ПО,1c.pdf,Установленные значения настроек сохраняются ме...,Logical thinking,"Как сохранить настройки отчета, чтобы использо...",Настройки отчета сохраняются через меню Все де...
74,Техническая документация,Руководства к РФ ПО,mysql.pdf,"При первичном запуске, необходимо ввести парол...",Logical thinking,Как изменить пароль главного администратора си...,Пароль главного администратора изменяется чере...
75,Техническая документация,Руководства к РФ ПО,mysql.pdf,Единовременное назначение пользователю только ...,Logical thinking,Можно ли назначить один прибор нескольким поль...,Нет. Программа позволяет назначить только один...
